# App Man

Here is my code : https://colab.research.google.com/drive/10CCDinuOinKl-0zr1opUWkVYI-IcJHZm?usp=sharing

## Library

In [ ]:
!pip install sentence_transformers -qq
!pip install -U gdown -qq
!pip install annoy -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 113.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 KB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from sklearn.metrics import f1_score
import gdown
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob

import cv2
from google.colab.patches import cv2_imshow
from PIL import Image, ImageOps
from skimage import io
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

import shutil

from scipy.spatial import distance
from annoy import AnnoyIndex
from natsort import natsorted

import imageio.core.util

## Load Data

In [ ]:
!gdown 1XaqA-9Y7EkRstmecKZ35SFKyf5EDaF2c #image
!gdown 1YdYGDjb_HNIAmjhhc7kfqGMPY3LsS3WN #validation

Downloading...
From: https://drive.google.com/uc?id=1XaqA-9Y7EkRstmecKZ35SFKyf5EDaF2c
To: /content/appman-image-search.zip
100% 36.2M/36.2M [00:01<00:00, 25.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YdYGDjb_HNIAmjhhc7kfqGMPY3LsS3WN
To: /content/validation.zip
100% 153k/153k [00:00<00:00, 21.5MB/s]


In [ ]:
!unzip -q /content/appman-image-search.zip
!unzip -q  /content/validation.zip

## Preprocessing

In [ ]:
!cp -r /content/queries /content/queries_2
!cp -r /content/test /content/test_2

In [ ]:
# preprocessing auto contrast
base = '/content/queries_2/queries'
for i in tqdm(os.listdir(base)):
  img_path = os.path.join(base, i)
  img = cv2.imread(img_path)
  img = cv2.resize(img, (224,224), interpolation=cv2.INTER_LANCZOS4)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = Image.fromarray(img)
  img = ImageOps.autocontrast(img) #สีเข้มขึ้น
  img.save(img_path)

100%|██████████| 22/22 [00:00<00:00, 29.92it/s]


In [ ]:
# preprocessing auto contrast
base = '/content/test_2/images'
for i in tqdm(os.listdir(base)):
  img_path = os.path.join(base, i)
  img = cv2.imread(img_path)
  img = cv2.resize(img, (224,224), interpolation=cv2.INTER_LANCZOS4)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = Image.fromarray(img)
  img = ImageOps.autocontrast(img) #สีเข้มขึ้น
  img.save(img_path)

100%|██████████| 1120/1120 [00:06<00:00, 166.98it/s]


In [ ]:
def normalize(image):
    mean = np.mean(image)
    std = np.std(image)
    normalized_img = (image - mean) / std
    return normalized_img

In [ ]:
#ทำให้เวลา save รูปของ skiimage ให้มันไม่ต้องโชว์ผลลัพธ์
def ignore_warnings(*args, **kwargs):
    pass

imageio.core.util._precision_warn = ignore_warnings

In [ ]:
#normalize data
base = '/content/queries_2/queries'
for i in tqdm(os.listdir(base)):
  img_path = os.path.join(base, i)
  image = io.imread(img_path)
  normalized_image = normalize(image)
  io.imsave(f"{img_path}", normalized_image)

100%|██████████| 22/22 [00:00<00:00, 163.48it/s]


In [ ]:
#normalize data
base = '/content/test_2/images'
for i in tqdm(os.listdir(base)):
  img_path = os.path.join(base, i)
  image = io.imread(img_path)
  normalized_image = normalize(image)
  io.imsave(f"{img_path}", normalized_image)

100%|██████████| 1120/1120 [00:06<00:00, 165.77it/s]


## submit kaggle

In [ ]:
submit = pd.read_csv('/content/sample_submission.csv')

path_queries = [path for path in glob('/content/queries_2/queries/*.jpg')]
path_queries = natsorted(path_queries, key=str)

#queries
queries = []
for f in tqdm(path_queries):
    a = Image.open(f)
    a = a.convert("L")
    queries.append(a)

#test
queries_t = []
base = '/content/test_2/images'
for f in tqdm(submit['img_file']):
  img_path = os.path.join(base, f)
  # img_path = f + '.jpg'
  a = Image.open(img_path)
  a = a.convert("L")
  queries_t.append(a)

100%|██████████| 1120/1120 [00:01<00:00, 853.75it/s]


In [ ]:
img_model = SentenceTransformer('clip-ViT-B-32')

embedded_queries = img_model.encode(queries)
embedded_queries = np.array(embedded_queries)

embedded_queries_t = img_model.encode(queries_t)
embedded_queries_t = np.array(embedded_queries_t)

In [ ]:
annoy_index = AnnoyIndex(512, 'manhattan')  

# add features to annoy_index
for i in tqdm(range(len(embedded_queries))):
    feature = embedded_queries[i]
    # Adds each feature vector to annoy index
    annoy_index.add_item(i, feature)

# Builds 99 search trees for the items added to index
annoy_index.build(99)

100%|██████████| 22/22 [00:00<00:00, 11029.73it/s]


True

In [ ]:
queries2 = queries.copy()
embedded_queries2 = embedded_queries.copy()

label_t = []
for i in tqdm(embedded_queries_t):
  a = annoy_index.get_nns_by_vector(i, 22, include_distances=True)
  thresh = a[1][0]
  if thresh < 117.38000000000378:
    label_t.append(a[0][0])
  else:
    label_t.append(22)

100%|██████████| 1120/1120 [00:00<00:00, 8838.70it/s]


In [ ]:
submit['class'] = label_t
submit.to_csv('kaggle_machima-2.csv')
submit.head()

,img_file,class
0,64ccfecf-e451-49a8-aa3f-acf2622a9a5c.jpg,12
1,c6df1385-382a-4428-b41e-f2d729b90c87.jpg,22
2,af30e9d0-da6e-42bd-814e-c70a0c16e554.jpg,22
3,3fc8998e-0324-426c-8233-6b76abc7e200.jpg,22
4,309d1085-0d11-4411-9555-b24cc8fcee02.jpg,22


## ส่งงาน validation

In [ ]:
validation = pd.read_csv('/content/final.csv')
validation

,name,count
0,0edb36a9-1be9-451f-889a-a8e38831a74e,NaN
1,0a9dce26-1ed4-4337-ad25-a80c9f1d8bcb,NaN
2,1cb1212d-3e8c-4e88-a011-fb1866770090,NaN
3,0b0eac7c-bf3c-4b34-b53c-f777a836348e,NaN
4,6e22539f-146c-44ad-ac67-76f0f1abc94a,NaN


In [ ]:
number = 22
for i in validation['name']:

  #auto contrast
  img_path = f'{i}.jpg'
  img = cv2.imread(img_path)
  img = cv2.resize(img, (224,224), interpolation=cv2.INTER_LANCZOS4)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = Image.fromarray(img)
  img = ImageOps.autocontrast(img) #สีเข้มขึ้น
  img.save(img_path)

  #normalize
  image = io.imread(img_path)
  normalized_image = normalize(image)
  io.imsave(f"{img_path}", normalized_image)

  #move folder to queries
  os.rename(f'{i}.jpg', f'{number}.jpg')
  shutil.move(f'{number}.jpg', "/content/queries_2/queries")
  number += 1

In [ ]:
submit = pd.read_csv('/content/sample_submission.csv')

path_queries = [path for path in glob('/content/queries_2/queries/*.jpg')]
path_queries = natsorted(path_queries, key=str)

#queries
queries = []
for f in tqdm(path_queries):
    a = Image.open(f)
    a = a.convert("L")
    queries.append(a)

#test
queries_t = []
base = '/content/test_2/images'
for f in tqdm(submit['img_file']):
  img_path = os.path.join(base, f)
  # img_path = f + '.jpg'
  a = Image.open(img_path)
  a = a.convert("L")
  queries_t.append(a)

100%|██████████| 1120/1120 [00:00<00:00, 1169.94it/s]


In [ ]:
%%time
embedded_queries = img_model.encode(queries)
embedded_queries = np.array(embedded_queries)

embedded_queries_t = img_model.encode(queries_t)
embedded_queries_t = np.array(embedded_queries_t)

CPU times: user 4.85 s, sys: 859 ms, total: 5.71 s
Wall time: 5.72 s


In [ ]:
queries2 = queries.copy()
embedded_queries2 = embedded_queries.copy()
annoy_index = AnnoyIndex(512, 'manhattan')  

# add features to annoy_index
for i in range(len(embedded_queries)):
    feature = embedded_queries[i]
    # Adds each feature vector to annoy index
    annoy_index.add_item(i, feature)

# Builds 40 search trees for the items added to index
annoy_index.build(99)

label_d = []
clas = 27
for i in tqdm(range(len(embedded_queries_t))):
  a = annoy_index.get_nns_by_vector(embedded_queries_t[i], clas, include_distances=True)
  thresh = a[1][0]
  if thresh < 117.38000000000378:
    label_d.append(a[0][0])
  else:
    clas += 1
    label_d.append(clas)
    embedded_queries2 = list(embedded_queries2)
    embedded_queries2.append(embedded_queries_t[i])
    embedded_queries2 = np.array(embedded_queries2)
    queries2.append(queries_t[i])

    annoy_index = AnnoyIndex(512, 'manhattan')  
    # add features to annoy_index
    for j in range(len(embedded_queries2)):
      feature = embedded_queries2[j]
      # Adds each feature vector to annoy index
      annoy_index.add_item(j, feature)

    # Builds 40 search trees for the items added to index
    annoy_index.build(99)

100%|██████████| 1120/1120 [00:01<00:00, 1065.64it/s]


In [ ]:
#count validation
[label_d.count(22), label_d.count(23), label_d.count(24), label_d.count(25), label_d.count(26)]

[15, 0, 4, 33, 2]

In [ ]:
validation['count'] = [label_d.count(22), label_d.count(23), label_d.count(24), label_d.count(25), label_d.count(26)]
validation

,name,count
0,0edb36a9-1be9-451f-889a-a8e38831a74e,15
1,0a9dce26-1ed4-4337-ad25-a80c9f1d8bcb,0
2,1cb1212d-3e8c-4e88-a011-fb1866770090,4
3,0b0eac7c-bf3c-4b34-b53c-f777a836348e,33
4,6e22539f-146c-44ad-ac67-76f0f1abc94a,2


In [ ]:
validation.to_csv('validation_machima-2.csv', index=False)